###  Testing with SDK v2

In [ ]:
# import required libraries
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.identity import DefaultAzureCredential
import sys

# Enter details of your AML workspace
subscription_id = "840b5c5c-3f4a-459a-94fc-6bad2a969f9d"
resource_group = "ml"
workspace = "ws02ent"
# get a handle to the workspace
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

sys.path.append("../") # go to parent dir

In [ ]:
from src.ray_on_aml.core import Ray_On_AML
from azure.ai.ml import  Input

import ray

ray_on_aml =Ray_On_AML(ml_client=ml_client, compute_cluster ="ds11" )
#For use as client mode, uncomment these lines
# head_ip = ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"])
# client = ray.init(f"ray://{head_ip}:10001")
#use CI as head node
ray_on_aml.getRay(ci_is_head=True, num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"])

In [ ]:
ray.cluster_resources()

In [ ]:
client.disconnect()

###  Testing with SDK v1

In [ ]:
ray_on_aml.shutdown(end_all_runs=True)

In [ ]:
from azureml.core import Workspace
import sys
sys.path.append("../") # go to parent dir
import importlib


In [ ]:
from src.ray_on_aml.core import Ray_On_AML
import ray
ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="ds11",maxnode=2)
ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"], ci_is_head=True)

# head_ip = ray_on_aml.getRay(num_node=2,pip_packages=["torch==1.13.0", "azureml-mlflow"], ci_is_head=True)

# client = ray.init(f"ray://{head_ip}:10001")


In [ ]:
ray.shutdown()

In [ ]:
ray.cluster_resources()

In [ ]:
ray_on_aml.shutdown()

In [ ]:
import ray

# Load data.
dataset = ray.data.read_csv("https://azuremlexamples.blob.core.windows.net/datasets/iris.csv").repartition(4)

train_dataset, valid_dataset = dataset.train_test_split(test_size=0.3)

# Create a test dataset by dropping the target column.
test_dataset = valid_dataset.drop_columns(cols=["target"])
# Create a preprocessor to scale some columns.

import numpy as np

from ray.data.preprocessors import Concatenator, Chain, StandardScaler

# Create a preprocessor to scale some columns and concatenate the result.
# preprocessor = Chain(
#     StandardScaler(columns=["mean radius", "mean texture"]),
#     Concatenator(exclude=["target"], dtype=np.float32),
# )


In [ ]:
pip install pyarrow==6.0.1

In [ ]:
import torch
import torch.nn as nn
from torch.nn.modules.utils import consume_prefix_in_state_dict_if_present

from ray import train
from ray.air import session
from ray.air.config import ScalingConfig
from ray.train.torch import TorchCheckpoint, TorchTrainer


def create_model(input_features):
    return nn.Sequential(
        nn.Linear(in_features=input_features, out_features=16),
        nn.ReLU(),
        nn.Linear(16, 16),
        nn.ReLU(),
        nn.Linear(16, 1),
        nn.Sigmoid(),
    )


def train_loop_per_worker(config):
    batch_size = config["batch_size"]
    lr = config["lr"]
    epochs = config["num_epochs"]
    num_features = config["num_features"]

    # Get the Ray Dataset shard for this data parallel worker,
    # and convert it to a PyTorch Dataset.
    train_data = train.get_dataset_shard("train")
    # Create model.
    model = create_model(num_features)
    model = train.torch.prepare_model(model)

    loss_fn = nn.BCELoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr)

    for cur_epoch in range(epochs):
        for batch in train_data.iter_torch_batches(
            batch_size=batch_size, dtypes=torch.float32
        ):
            # "concat_out" is the output column of the Concatenator.
            inputs, labels = batch["concat_out"], batch["target"]
            optimizer.zero_grad()
            predictions = model(inputs)
            train_loss = loss_fn(predictions, labels.unsqueeze(1))
            train_loss.backward()
            optimizer.step()
        loss = train_loss.item()
        session.report({"loss": loss}, checkpoint=TorchCheckpoint.from_model(model))


num_features = len(train_dataset.schema().names) - 1

trainer = TorchTrainer(
    train_loop_per_worker=train_loop_per_worker,
    train_loop_config={
        "batch_size": 128,
        "num_epochs": 20,
        "num_features": num_features,
        "lr": 0.001,
    },
    scaling_config=ScalingConfig(
        num_workers=2,  # Number of workers to use for data parallelism.
        use_gpu=False,
        trainer_resources={"CPU": 1},  # so that the example works on Colab.
    ),
    datasets={"train": train_dataset},
    preprocessor=preprocessor,
)
# Execute training.
result = trainer.fit()
print(f"Last result: {result.metrics}")
# Last result: {'loss': 0.6559339960416158, ...}

In [ ]:
pip install --upgrade ray

In [ ]:
ray_on_aml.shutdown()

In [ ]:
import ray
client = ray.init(f"ray://{head_ip}:10001")


In [ ]:
client

In [ ]:
ray.shutdown()

In [ ]:
client.disconnect()